In [ ]:
import numpy as np
import pandas as pd
import json

fine_path = 'data/train_fine.txt'
dic = set()
count = 1
l = 0
with open(fine_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        l = max(0, len(data['title']))
        for key in data['key_attr']:
            if key not in dic:
                dic.add(key)
                count += 1

print(key)
print(dic)


In [ ]:
print(l)

In [3]:
import collections
import json
key_dic = {}
i = 0
with open('data/attr_to_attrvals.json', 'r', encoding='utf-8') as f:
    key_attr = json.load(f)
    for key in key_attr:
        key_dic[key] = i
        i += 1
print(key_dic)


{'领型': 0, '袖长': 1, '衣长': 2, '版型': 3, '裙长': 4, '穿着方式': 5, '类别': 6, '裤型': 7, '裤长': 8, '裤门襟': 9, '闭合方式': 10, '鞋帮高度': 11}


In [ ]:
import os 
import numpy as np
import json
import random
import copy

data_path_1 = './data/train_fine.txt'
data_path_2 = './data/train_coarse.txt'

data_pos_list = []
data_neg_list = []

with open(data_path_1, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

with open(data_path_2, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

new_neg_list = []
for i in range(len(data_pos_list)):
    dic = copy.deepcopy(data_pos_list[i])
    index = random.randint(0, len(data_pos_list)-1)
    while index == i:
        index = random.randint(0, len(data_pos_list)-1)
    dic['title'] = data_pos_list[index]['title']
    dic['match']['图文'] = 0
    new_neg_list.append(dic)

print(len(data_pos_list), len(data_neg_list), len(new_neg_list))
all_data = data_pos_list + data_neg_list + new_neg_list
np.random.shuffle(all_data)
l = int(len(all_data) * 0.9)
x_train_list = all_data[:l]
x_val_list = all_data[l:]
# np.random.shuffle(x_val_list)

x_train_list = [json.dumps(dic, ensure_ascii=False)+'\n' for dic in x_train_list]
x_val_list = [json.dumps(dic, ensure_ascii=False)+'\n' for dic in x_val_list]


# with open('./data/pretrain_match.txt', 'w', encoding='utf-8') as f:
#     f.writelines(pre_ret)

with open('./data/new_train_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(x_train_list)

with open('./data/new_val_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(x_val_list)

# print(len(pre_ret))
print(len(x_train_list))
print(len(x_val_list))
print(len(x_train_list) + len(x_val_list))

In [ ]:
import json
new_dic = {}

with open('./data/attr_to_attrvals.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)
    for key, value in attr_key.items():
        tmp = []
        for v in value:
            if '=' in v:
                tmp.append(v.split('='))
            else:
                tmp.append([v])
        new_dic[key] = tmp
print(new_dic)
rets = [json.dumps(new_dic, ensure_ascii=False)+'\n']
with open('./data/attr_match.json', 'w', encoding='utf-8') as f:
    f.writelines(rets)

In [4]:
import json
import numpy as np
not_math_list = []

with open('./data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

def get_title_mask(title, key, val, attr_key):
    values = attr_key[key]
    key_index = 0
    for i in range(len(values)):
        if val in values[i]:
            key_index = i
            break
    new_index = np.random.randint(len(values))
    while new_index == key_index:
        new_index = np.random.randint(len(values))
    sub_val = values[new_index]
    new_sub_val = sub_val[np.random.randint(len(sub_val))]

    return title.replace(val, new_sub_val, 1)

def get_random_key(keys, ratio=0.77):
    l = int(len(keys) * ratio)
    if l == 0:
        l = 1
    np.random.shuffle(keys)
    return keys[:l]

count = 0
with open('./data/train_fine.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    new_dic = {}
    for line in lines:
        data = json.loads(line)
        new_dic = data
        
        keys = get_random_key([x for x in new_dic['match'].keys() if x != '图文'])
        for key in keys:
            new_dic['title'] = get_title_mask(new_dic['title'], key, new_dic['key_attr'][key], attr_key)
            new_dic['match'][key] = 0
        not_math_list.append(json.dumps(new_dic, ensure_ascii=False)+'\n')
        print(new_dic['match'])
        
with open('./data/neg_fine.txt', 'w', encoding='utf-8') as f:
    f.writelines(not_math_list)

        

{'图文': 1, '裤型': 1, '裤长': 0}
{'图文': 1, '版型': 0, '穿着方式': 1}
{'图文': 1, '版型': 1, '袖长': 0, '领型': 0}
{'图文': 1, '领型': 0}
{'图文': 1, '版型': 1, '衣长': 0, '袖长': 0}
{'图文': 1, '版型': 0, '衣长': 1, '领型': 0}
{'图文': 1, '裤门襟': 0}
{'图文': 1, '版型': 0, '袖长': 0, '领型': 1}
{'图文': 1, '裤型': 1, '裤长': 0}
{'图文': 1, '衣长': 1, '袖长': 0, '领型': 0}
{'图文': 1, '衣长': 1, '领型': 0}
{'图文': 1, '衣长': 0, '领型': 1}
{'图文': 1, '版型': 1, '衣长': 0, '袖长': 0, '领型': 0}
{'图文': 1, '领型': 0}
{'图文': 1, '闭合方式': 0, '鞋帮高度': 1}
{'图文': 1, '版型': 1, '领型': 0}
{'图文': 1, '版型': 0, '袖长': 1}
{'图文': 1, '闭合方式': 1, '鞋帮高度': 0}
{'图文': 1, '裤长': 0}
{'图文': 1, '版型': 0, '衣长': 0, '领型': 1}
{'图文': 1, '裤门襟': 0}
{'图文': 1, '版型': 1, '袖长': 0}
{'图文': 1, '袖长': 0}
{'图文': 1, '闭合方式': 0, '鞋帮高度': 1}
{'图文': 1, '版型': 0, '衣长': 0, '领型': 1}
{'图文': 1, '穿着方式': 0, '袖长': 1}
{'图文': 1, '裙长': 1, '领型': 0}
{'图文': 1, '版型': 0, '衣长': 1, '袖长': 0, '领型': 0}
{'图文': 1, '版型': 1, '袖长': 0, '领型': 0}
{'图文': 1, '版型': 0, '穿着方式': 1, '领型': 0}
{'图文': 1, '版型': 1, '袖长': 0}
{'图文': 1, '类别': 0}
{'图文': 1, '版型': 0, '穿着方式': 0, 

In [5]:
print(len(not_math_list))

50000


In [2]:
import torch
import numpy as np
a = np.zeros((6, 12))
a[1][0] = 1
a[1][4] = 1
a[1][5] = 1
b = torch.from_numpy(a)
c = b[1][0,4,3]
c

IndexError: too many indices for tensor of dimension 1